In [ ]:
# 多图round-i 数据提取和合并
import json
import os
import os.path as osp
from tqdm import tqdm
generated_and_corrected_qa = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/mosaic_imgs/data/multi_round_qa_v1_mosaic_imgs_3k/round_1/gen_qa_corrected"

mosaic_files = os.listdir(generated_and_corrected_qa)

count = 0
for mfs in tqdm(mosaic_files):
    collection = {
        "top left":    [],
        "top right":   [],
        "lower left":  [],
        "lower right": []
    }
    try:
        path = osp.join(generated_and_corrected_qa, mfs)
        data = json.load(open(path, "r"))

        for d in data:
            collection[d["image_position"]].append(d)
        pos2id = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/evolution/mosaic_imgs/data/meta_qa_det_cap/round1", mfs), "r"))["id2loc"]
        
        for pos, gen_qa in collection.items():
            json.dump(gen_qa, open(osp.join("/mnt/workspace/workgroup/haonan/evolution-codeution/mosaic_imgs/data/multi_round_qa_v1_mosaic_imgs_3k/round_1/imgs_qa", "{}.json".format(pos2id[pos])), "w"), indent=4)
            count += 1
    except:
        continue
print("{}/{} valid imgage qa".format(count, len(mosaic_files)*4))

In [1]:
# 演化前后打分
import os.path as osp
import os
import json
data_list = [
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k/round1/score_gpt4_mini_corrected",
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k/round2/score_gpt4_mini_corrected",
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k/round3/score_gpt4_mini_corrected",
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k_mini/round1/score_gpt4_mini_corrected",
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k_mini/round2/score_gpt4_mini_corrected",
             "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k_mini/round3/score_gpt4_mini_corrected",
             ]
# data_list = ["/mnt/workspace/workgroup/haonan/evolution-code/evolution/mosaic_imgs/data/multi_round_qa_v1_mosaic_imgs_3k/round_1/score_corrected"]
for dl in data_list:
    score_file = os.listdir(dl)
    improved_num = 0.
    total_number = 0.
    total_score = 0.

    for sf in score_file:
        file = osp.join(dl, sf)
        try:
            # print(file)
            data = json.load(open(file, "r"))
            total_number += len(data)
            for i in data:
                total_score += i['score'] if i["improved"] == "yes" else 0
                improved_num += 1 if i["improved"] == "yes" else 0
        except:
            continue
    # print(improved_num, total_number, total_score)
    print("average improving ratio: {:.2f}, average score: {:.2f}, improved samples: {}, total samples: {}".format(improved_num / total_number, total_score / improved_num, improved_num, total_number))     

average improving ratio: 0.72, average score: 6.19, improved samples: 20654.0, total samples: 28654.0
average improving ratio: 0.87, average score: 6.68, improved samples: 19789.0, total samples: 22790.0
average improving ratio: 0.92, average score: 7.01, improved samples: 18873.0, total samples: 20527.0
average improving ratio: 0.68, average score: 6.06, improved samples: 18110.0, total samples: 26647.0
average improving ratio: 0.85, average score: 6.59, improved samples: 17117.0, total samples: 20219.0
average improving ratio: 0.91, average score: 6.95, improved samples: 16399.0, total samples: 18075.0


In [19]:
# 每个方向演化的样本数量
import json
import os.path as osp
count = {
    "bf_in_breath": 0,
    "df_cot": 0, 
    "df_format": 0
}
g = 0
evo = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/evo_path.json", "r"))
# print(len(evo)) # 484
for k, v in evo.items():
    file = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round3/filtered_qa", k + ".json"), "r"))
    g += len(file["conversations_v0"])
    for index, vv in enumerate(v):
        count[vv] += len(file["conversations_v{}".format(index+1)])
count, sum(count.values())

({'bf_in_breath': 1299, 'df_cot': 962, 'df_format': 546}, 2807)

In [ ]:
# score 和 演化qa数量不一致统计
import json
import os.path as osp
import os

a = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round1/gen_qa_corrected")
b = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round1/score_gpt4_mini_corrected")
print(len(a), len(b))
c = 0
n1 = 0
n2 = 0
v = 0
for i in a:
    try:
        # g = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round1/gen_qa_corrected/"+i, "r"))
        y = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round1/score_gpt4_mini_corrected/"+i, "r"))
        # if len(g) != len(y):
        #     c+=1
        # n1 += len(g)
        # n2 += len(y)
    except:
        v+=1
        print(i)
        continue
n1, n2, v

In [4]:
# coco_1k 数据重构
import json
import os
import os.path as osp
import shutil
from tqdm import tqdm
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_30k_addition_science.json", "r"))

# datasets_path = "/mnt/workspace/workgroup/haonan/evolution-code/datasets"
# if os.path.exists(osp.join(datasets_path, "meta_data")):
#     shutil.rmtree(osp.join(datasets_path, "meta_data"))
#     os.mkdir(osp.join(datasets_path, "meta_data"))

data_v2 = []
for d in tqdm(data):
    qa = []
    for qa_index in range(len(d['conversations_v0'])//2):
        qa.append(
            {
                "image_position": "single",
                "objects": [],
                "skills": [],
                "format": "Norm",
                "question": d['conversations_v0'][2*qa_index]['value'].replace("<image>\n", "").replace("\n<image>", "").strip(),
                "steps": [],
                "answer": d['conversations_v0'][2*qa_index+1]['value'].replace("<image>\n", "").replace("\n<image>", "").strip() # provided only for format
            }
        )

    tmp = {
        "id": d["id"],
        "image": d["image"],
        "conversations_v0": qa,
        "topic_name": d["topic_name"] if "topic_name" in d else [],
        "caption": d["caption"],
        "det": d["det"]

    }
    data_v2.append(tmp)


json.dump(data_v2, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_30k_addition_science_v2.json", "w"), indent=4)

100%|██████████| 29032/29032 [00:01<00:00, 20360.00it/s]


In [13]:
# 补充seed样本中缺失的部分
import json
import os
import os.path as osp
import shutil
from tqdm import tqdm
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_complementary.json", "r"))

c = 0
id_list = []
for d in tqdm(data):
    if len(d["conversations_v0"]) == 0:
        c+=1
        id_list.append(d["id"])
        try:
            qa = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_qa_single_53k/ini_qa_v1_single_img_53k_corrected", "{}.json".format(d["id"]))))
            lll = []
            for s in qa:
                tmp = {
                    "image_position": "single",
                    "objects": s["objects"],
                    "skills": s["skills"],
                    "format": "Norm",
                    "question": s["question"],
                    "steps": s["steps"],
                    "answer": s["answer"]
                }
                lll.append(tmp)
            d["conversations_v0"] = lll
            c+=1
        except:
            continue
id_list
# json.dump(data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_complementary.json", "w"), indent=4)

100%|██████████| 29032/29032 [00:00<00:00, 1107152.26it/s]


In [2]:
# 构造hash_id建立唯一索引，因为id和image键值都存在重复值
import json

a = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_gpt4o_evo_add_60-50-10k_2.json", "r"))
for index, i in enumerate(a):
    # i["hash_id"] = str(index) + "_" + i["id"] + "_" + i["image"].replace("/", "_")
    i["hash_id"] = str(index) + "_" + i["image"].replace("/", "_")

json.dump(a, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_gpt4o_evo_add_60-50-10k_2.json", "w"), indent=4)

In [1]:
import json
d = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_gpt4o_evo_add_60-50k.json", "r"))
json.dump(d[10000:20000], open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_gpt4o_evo_add_60-50-10k_2.json", "w"))

In [3]:
# 如果数据格式已经组织完善，直接分开存到meta data里
import os
import json
import os.path as osp
from tqdm import tqdm
import shutil
datasets_path = "/mnt/workspace/workgroup/haonan/evolution-code/datasets"
if os.path.exists(osp.join(datasets_path, "meta_data_gpt4o_10k")):
    shutil.rmtree(osp.join(datasets_path, "meta_data_gpt4o_10k"))
    os.mkdir(osp.join(datasets_path, "meta_data_gpt4o_10k"))

data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_gpt4o_evo_add_60-50-10k.json", "r"))

for index, d in enumerate(tqdm(data)):
    json.dump(d, open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data_gpt4o_10k", "{}.json".format(d["hash_id"])), "w"), indent=4)
    # json.dump(d, open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data", "{}.json".format(d["id"])), "w"), indent=4)


100%|██████████| 10000/10000 [01:43<00:00, 96.98it/s]


In [4]:
import os
print(len(os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data_additional_data")))

29032


In [ ]:
# 额外的图表数据id会重复，找出重复id的样本重新生成
print(len(re))
ss = []
for k, v in re.items():
    if v > 1:
        ss.append(k)
ss
# json.dump(ss, open("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_objects_single_repeated_id_science_2k/id.json", "w"))

In [4]:
import os
print(len(os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data_additional_data")))

27227


In [1]:
# 合并conversation_v1
import os 
import os.path as osp
import shutil
import json
from tqdm import tqdm

meta_data = "/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data"
conversation_v1_path = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_ini_prompt/round1"
# conversation_v1_files = os.listdir(osp.join(conversation_v1_path, "gen_qa_corrected"))
meta_data_files = os.listdir(meta_data)

if os.path.exists(osp.join(conversation_v1_path, "filtered_qa")):
    shutil.rmtree(osp.join(conversation_v1_path, "filtered_qa"))
    os.mkdir(osp.join(conversation_v1_path, "filtered_qa"))

count = 0
for mdf in tqdm(meta_data_files):

    meta_sample = json.load(open(osp.join(meta_data, mdf), "r"))
    qa_path = osp.join(conversation_v1_path, "gen_qa_corrected", mdf)
    score_path = osp.join(conversation_v1_path, "score_gpt4_mini_corrected", mdf)
    
    try:
        data = json.load(open(qa_path, "r"))
        score = json.load(open(score_path, "r"))

        temp = []
        for index, i in enumerate(data):
            if score[index]['improved'] == "yes":
                i["score"] =  score[index]['score']
                temp.append(i)

        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": temp,
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v1_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
        count += 1
    except:
        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": [],
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v1_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
print("{} of {} is evolved".format(count, len(meta_data_files)))

100%|██████████| 977/977 [00:08<00:00, 110.87it/s]

950 of 977 is evolved


In [1]:
# 合并conversation_v2
import os 
import os.path as osp
import json
from tqdm import tqdm
import shutil

meta_data = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_ini_prompt/round1/filtered_qa"
conversation_v2_path = "/mnt/workspace/workgroup/haonan/evolution-code/single_imgs/multi_round_v1_single_imgs_ini_prompt/round2"
# conversation_v1_files = os.listdir(osp.join(conversation_v1_path, "gen_qa_corrected"))
meta_data_files = os.listdir(meta_data)

if os.path.exists(osp.join(conversation_v2_path, "filtered_qa")):
    shutil.rmtree(osp.join(conversation_v2_path, "filtered_qa"))
    os.mkdir(osp.join(conversation_v2_path, "filtered_qa"))

count = 0
for mdf in tqdm(meta_data_files):

    meta_sample = json.load(open(osp.join(meta_data, mdf), "r"))
    qa_path = osp.join(conversation_v2_path, "gen_qa_corrected", mdf)
    score_path = osp.join(conversation_v2_path, "score_gpt4_mini_corrected", mdf)
    
    try:
        data = json.load(open(qa_path, "r"))
        score = json.load(open(score_path, "r"))
        temp = []
        for index, i in enumerate(data):
            if score[index]['improved'] == "yes":
                i["score"] =  score[index]['score']
                temp.append(i)

        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": meta_sample["conversations_v1"],
            "conversations_v2": temp,
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v2_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
        count += 1
    except:
        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": meta_sample["conversations_v1"],
            "conversations_v2": [],
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v2_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
print("{} of {} is evolved".format(count, len(meta_data_files)))

100%|██████████| 977/977 [00:07<00:00, 125.62it/s]

974 of 977 is evolved


In [3]:
# 合并conversation_v3
import os 
import os.path as osp
import json
import shutil
from tqdm import tqdm

meta_data = "/mnt/workspace/workgroup/haonan/evolution-codeution/single_imgs/multi_round_v1_single_imgs_ini_prompt/round2/filtered_qa"
conversation_v3_path = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_ini_prompt/round3"
# conversation_v1_files = os.listdir(osp.join(conversation_v1_path, "gen_qa_corrected"))
meta_data_files = os.listdir(meta_data)

if os.path.exists(osp.join(conversation_v3_path, "filtered_qa")):
    shutil.rmtree(osp.join(conversation_v3_path, "filtered_qa"))
    os.mkdir(osp.join(conversation_v3_path, "filtered_qa"))

count = 0

for mdf in tqdm(meta_data_files):

    meta_sample = json.load(open(osp.join(meta_data, mdf), "r"))
    qa_path = osp.join(conversation_v3_path, "gen_qa_corrected", mdf)
    score_path = osp.join(conversation_v3_path, "score_gpt4_mini_corrected", mdf)
    
    try:
        data = json.load(open(qa_path, "r"))
        score = json.load(open(score_path, "r"))
        temp = []
        for index, i in enumerate(data):
            if score[index]['improved'] == "yes":
                i["score"] =  score[index]['score']
                temp.append(i)

        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": meta_sample["conversations_v1"],
            "conversations_v2": meta_sample["conversations_v2"],
            "conversations_v3": temp,
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v3_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
        count += 1
    except:
        d = {
            "id": meta_sample["id"],
            "image": meta_sample["image"],
            "conversations_v0": meta_sample["conversations_v0"],
            "conversations_v1": meta_sample["conversations_v1"],
            "conversations_v2": meta_sample["conversations_v2"],
            "conversations_v3": [],
            "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
            "caption": meta_sample["caption"],
            "det": meta_sample["det"]
        }
        json.dump(d, open(osp.join(conversation_v3_path, "filtered_qa", "{}.json".format(meta_sample['id'])), "w"), indent=4)
print("{} of {} is evolved".format(count, len(meta_data_files)))

  0%|          | 0/977 [00:00<?, ?it/s]

100%|██████████| 977/977 [00:08<00:00, 118.97it/s]

976 of 977 is evolved


In [5]:
# 合并成训练文件
import os.path as osp
import json
import os
from tqdm import tqdm

data_path = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_ini_prompt/round3/filtered_qa"
data_file_list = os.listdir(data_path)

merged_data = []
for data_file in tqdm(data_file_list):
    data_file_path = osp.join(data_path, data_file)
    data = json.load(open(data_file_path, "r"))
    merged_data.append(data)

json.dump(merged_data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/coco_instruct_ablation_topic_25_1k_v2_3round_evo_ini_prompt.json", "w"), indent=4)

 57%|█████▋    | 557/977 [00:00<00:00, 1363.67it/s]

100%|██████████| 977/977 [00:00<00:00, 1257.49it/s]


In [ ]:
# 演化路径合并
import json
import os
import os.path as osp

root_path = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_persona/round{}"

files = os.listdir(osp.join(root_path.format("1"), "evo_path"))
print(len(files))
data = {}

for f in files:
    conv1 = json.load(open(osp.join(root_path.format("1"), "evo_path", f), "r"))
    conv2 = json.load(open(osp.join(root_path.format("2"), "evo_path", f), "r"))
    conv3 = json.load(open(osp.join(root_path.format("3"), "evo_path", f), "r"))

    data[f.replace(".json", "")] = [conv1, conv2, conv3]  
json.dump(data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/evo_path.json", "w"), indent=2)

In [4]:
# 打分情况合并
import json
import os
import os.path as osp
from tqdm import tqdm

root_path = "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_single_imgs_6k_mini/round{}"

files = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data_6k")

data = {}

for f in tqdm(files):

    conv0 = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data_6k", f), "r"))
    try:
        conv1 = json.load(open(osp.join(root_path.format("1"), "score_gpt4_mini_corrected", f), "r"))
    except:
        conv1 = []
    
    try:
        conv2 = json.load(open(osp.join(root_path.format("2"), "score_gpt4_mini_corrected", f), "r"))
    except:
        conv2 = []
    
    try:
        conv3 = json.load(open(osp.join(root_path.format("3"), "score_gpt4_mini_corrected", f), "r"))
    except:
        conv3 = []

    data[f.replace(".json", "")] = {
        "conversation_v0": conv0,
        "conversation_v1": conv1,
        "conversation_v2": conv2,
        "conversation_v3": conv3,
    }

json.dump(data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/improv_score_reason_6k_mini.json", "w"), indent=2)

100%|██████████| 6486/6486 [00:21<00:00, 304.14it/s]


In [1]:
# persona数据topk
import json
import os.path as osp
import os
import jsonlines

persona_data = []
with open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/persona.jsonl", "r") as file:
    for d in jsonlines.Reader(file):
        persona_data.append(d['persona'])
    
len(persona_data)

200000

In [2]:
import json

query_list = []
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/coco_instruct_ablation_topic_25_1k_add_v2.json", "r"))
for d in data:
    if len(d['caption']) != 0:
        query_list.append(d['caption'][1]["value"])
    else:
        sen = ''
        for index in range(len(d['conversations_v0'])):
            if index > 5: break
            sen += d['conversations_v0'][index]['question'] + " " + d['conversations_v0'][index]['answer'] + " "
        query_list.append(sen)

len(query_list)

1612

In [11]:
import os 
import os.path as osp
import json

files = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data")
dd = {}
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/coco_instruct_ablation_topic_25_1k_add_v2.json", "r"))

for index, d in enumerate(data):
    dd[d["id"]] = {
        "caption": d["caption"],
        "persona": [persona_data[i] for i in indice[index]]
    }

json.dump(dd, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/persona_top50.json", "w"), indent=4)

In [26]:
# 批量rename
import os
from tqdm import tqdm
path = '/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_qa_single_53k/ini_qa_v1_single_img_53k'
files = os.listdir(path)
for i, file in tqdm(enumerate(files)):
    NewName = os.path.join(path, file.replace("gen_qa_", ""))
    OldName = os.path.join(path, file)
    # print(NewName, OldName)
    os.rename(OldName, NewName)

In [25]:
# 合并1k的非采样数据
import json
import os
import os.path as osp
from tqdm import tqdm
import shutil

meta_data = "/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data"
gen_data = [
    "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/cot_qa_v1_single_imgs_1k/round1",
    "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/format_qa_v1_single_imgs_1k/round1",
    "/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/in_breath_qa_v1_single_imgs_1k/round1"
]
merged_data = []
# conversation_v1_files = os.listdir(osp.join(conversation_v1_path, "gen_qa_corrected"))
meta_data_files = os.listdir(meta_data)

count = 0
for mdf in tqdm(meta_data_files):
    meta_sample = json.load(open(osp.join(meta_data, mdf), "r"))
    temp = []
    for gen_qa_path in gen_data:
        qa_path = osp.join(gen_qa_path, "gen_qa_corrected", mdf)
        score_path = osp.join(gen_qa_path, "score_gpt4_mini_corrected", mdf)
        # print(mdf)
        try:
            data = json.load(open(qa_path, "r"))
            score = json.load(open(score_path, "r"))
            for index, i in enumerate(data):
                if score[index]['improved'] == "yes":
                    i["score"] =  score[index]['score']
                    temp.append(i)
        except:
            continue
    d = {
        "id": meta_sample["id"],
        "image": meta_sample["image"],
        "conversations_v0": meta_sample["conversations_v0"],
        "conversations_v1": temp,
        "topic_name": meta_sample["topic_name"] if "topic_name" in meta_sample else [],
        "caption": meta_sample["caption"],
        "det": meta_sample["det"]
    }
    merged_data.append(d)

json.dump(merged_data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/coco_instruct_ablation_topic_25_1k_parallel_3directions_evo_mini_score.json", "w"), indent=4)

100%|██████████| 977/977 [00:04<00:00, 210.92it/s]


In [9]:
# 检查objects/skills/steps为空的比例
import json
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_136k_mini.json", "r"))

count1 = 0
count2 = 0
count3 = 0

total1 = 0
total2 = 0
total3 = 0

for i in data:
    for j in i["conversations_v1"]:
        if len(j["objects"]) == 0:
            count1 += 1
    for j in i["conversations_v2"]:
        if len(j["objects"]) == 0:
            count2 += 1
    for j in i["conversations_v3"]:
        if len(j["objects"]) == 0:
            count3 += 1

    total1 += len(i["conversations_v1"])
    total2 += len(i["conversations_v2"])
    total3 += len(i["conversations_v3"])

print(count1, count2, count3)
print(total1, total2, total3)

27164 16576 11149
369362 361735 349303


In [1]:
# 统计qa对中objects/skills/steps缺失的比例
import json
seed_all_com = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_30k_addition_science_v2.json", "r"))
print(len(seed_all_com))
count0 = 0
total0 = 0
for i in seed_all_com:
    for j in i["conversations_v0"]:
        if len(j["objects"]) == 0:
            count0 += 1

    total0 += len(i["conversations_v0"])

print(count0)
print(total0)

29032
178554
178554


In [6]:
# 把生成的objects/skills/steps按照question是否一致进行填充
import os.path as osp
from tqdm import tqdm
import json

map_ = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_objects_single_repeated_id_science_2k/id.json", "r"))

c = 0
for i in tqdm(seed_all_com):
    # if len(i["conversations_v0"][0]["objects"]) != 0:
    #     continue
    file_id = i["id"]
    if file_id in map_:
        file_id = i["image"].replace("/", "_")
        try:
            gen_data = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_objects_single_repeated_id_science_2k/ini_object_v1_single_img_2k_corrected", file_id + ".json"), "r"))
        except:
            c+=1
            continue
        q_map = {}
        for h in gen_data:
            try:
                q_map[h["question"].lower().strip()] = h
            except:
                continue

        for x in i["conversations_v0"]:
            if len(x["objects"]) != 0:
                continue
            if x["question"].lower().strip() in q_map:
                q = x["question"].lower().strip()
                x["objects"] = q_map[q]["objects"]
                x["skills"] = q_map[q]["skills"]
                x["steps"] = q_map[q]["steps"]
    else:
        try:
            gen_data = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/gen_objects_single_153k/ini_object_v1_single_img_153k_corrected", file_id + ".json"), "r"))
        except:
            c+=1
            continue
        q_map = {}
        for h in gen_data:
            try:
                q_map[h["question"].lower().strip()] = h
            except:
                continue

        for x in i["conversations_v0"]:
            if len(x["objects"]) != 0:
                continue
            if x["question"].lower().strip() in q_map:
                q = x["question"].lower().strip()
                x["objects"] = q_map[q]["objects"]
                x["skills"] = q_map[q]["skills"]
                x["steps"] = q_map[q]["steps"]
c


  0%|          | 0/29032 [00:00<?, ?it/s]

100%|██████████| 29032/29032 [00:56<00:00, 512.82it/s]


2775

In [7]:
json.dump(seed_all_com, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_30k_addition_science_v2_add_obj_v3.json", "w"), indent=4)

In [9]:
# check和filter dataengine的脏图表数据
import json
import os 
from tqdm import tqdm
paths = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/dataengine_match_161k/gen_yes_no_id_more_than_one")

In [10]:
yes = 0
no = 0
for p in tqdm(paths):
    with open("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/dataengine_match_161k/gen_yes_no_id_more_than_one/" + p, 'r', encoding='utf-8') as file:
        content = file.read()    
    if content.lower().strip() == "yes":
        yes += 1
    elif content.lower().strip() == "no":
        no+=1

print(yes/len(paths))

100%|██████████| 98197/98197 [04:02<00:00, 405.51it/s]

0.2097314581911871


In [1]:
import jsonlines
data = {}
c = 0
ss = []
with open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/dataengine_161k.jsonl", "r") as file:
    for item in jsonlines.Reader(file):
        ss.append(item["id"])
        if item["id"] in data:
            data[item["id"]] += 1
        else:
            data[item["id"]] = 1

print(len(list(set(ss))))

97049


In [4]:
map_list = 0
for k, v in data.items():
    if v > 1:
        map_list+=v
map_list

101402

In [21]:
print(map_list[:5])
import json
json.dump(map_list, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/dataengine_id_list_more_than_one.json", "w"))

['data_engine_2', 'data_engine_3', 'data_engine_4', 'data_engine_5', 'data_engine_6']


In [5]:
import json
import os
import jsonlines
import os.path as osp
from tqdm import tqdm

data_path = "/mnt/workspace/workgroup/haonan/evolution-code/datasets/dataengine_161k.jsonl"

map_list = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/dataengine_id_list_more_than_one.json", "r"))

filted_data = []
data = []
with open(data_path, "r") as file:
    for item in jsonlines.Reader(file):
        data.append(item)

repeated_data = os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/dataengine_match_161k/gen_yes_no_id_more_than_one")

for rd in tqdm(repeated_data):
    with open("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/dataengine_match_161k/gen_yes_no_id_more_than_one/" + rd, 'r', encoding='utf-8') as file:
        content = file.read()    
    if content.lower().strip() == "yes":
        filted_data.append(data[int(rd.replace(".json", ""))])

with open(data_path, "r") as file:
    for item in jsonlines.Reader(file):
        if item['id'] not in map_list:
            try:
                with open("/mnt/workspace/workgroup/haonan/evolution-code/initial_stage/dataengine_match_161k/gen_yes_no/" + item['id'] + ".json", 'r', encoding='utf-8') as file:
                    content = file.read() 
                    if content.lower().strip() == "yes":
                        filted_data.append(item)
            except:
                continue
len(filted_data)

100%|██████████| 98197/98197 [02:04<00:00, 786.58it/s]


20839

In [10]:
# 挑可视化的样本
import json
from tqdm import tqdm
d = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_30k_add_mini.json", "r"))
sss = []
for i in tqdm(d):
    if len(i["conversations_v1"]) != 0 and len(i["conversations_v2"]) != 0 and len(i["conversations_v3"]) != 0:
        if i["conversations_v0"][0]["format"] == "Norm" and i["conversations_v1"][0]["format"] == "Norm" and i["conversations_v2"][0]["format"] != "Norm" and i["conversations_v2"][0]["format"] != "complex reasoning" and i["conversations_v3"][0]["format"] == "complex reasoning": 
            _ = i.pop("caption")
            _ = i.pop("det")
            _ = i.pop("topic_name")

            sss.append(i)

json.dump(sss, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/sample_selection.json", "w"), indent=2)

100%|██████████| 26794/26794 [00:00<00:00, 318382.06it/s]


In [4]:
# 合并最终全量数据
import json

a = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_15k_add_mini.json", "r"))
b = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_15k_mini.json", "r"))
c = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_30k_add_mini.json", "r"))
d = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_75-15_add_mini.json", "r"))
e = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_all_add_75k_mini.json", "r"))


13399 16277 26794 13399 66704


In [6]:
print(len(a)+len(b)+len(c)+len(d)+len(e))

136573


In [3]:
import os 
print(len(os.listdir("/mnt/workspace/workgroup/haonan/evolution-code/datasets/meta_data")))

9992


In [8]:
f = a+b+c+d+e
print(len(f))
json.dump(f, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_136k_mini.json", "w"))

136573


In [1]:
import json
dd = json.load(open("/mnt/workspace/workgroup/lr/data_haonan/seed_data_30k_addition_science_v2_add_obj_v3_mini.json", "r"))
ff = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_136k_mini.json", "r"))
print(len(dd), len(ff))

29032 136573


In [2]:
aa = dd + ff
print(len(aa))

165605


In [3]:
json.dump(aa, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_165k_mini.json", "w"))

In [24]:
import random
import numpy as np


index = range(len(aa))
fff = np.random.choice(index, 50000, replace=False)
hhh = [aa[i] for i in fff]
print(len(hhh))
json.dump(hhh, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_165k_mini_50k_for_chart_vis.json", "w"))

50000


In [19]:
import json
import os.path as osp
import os

data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/improv_score_reason_old.json", "r"))

for k, v in data.items():
    # try:
    v["conversation_v0"]["score"] = json.load(open(osp.join("/mnt/workspace/workgroup/haonan/evolution-code/evolution/single_imgs/multi_round_v1_single_imgs_ini_prompt_v0_score/round0/score_gpt4_mini_corrected", k+".json"), "r"))
    # except:
    #     continue

json.dump(data, open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/improv_score_reason_old_v0_score.json", "w"), indent=4)

In [1]:
import json
data = json.load(open("/mnt/workspace/workgroup/haonan/evolution-code/datasets/seed_data_all_136k_mini.json", "r"))

json.dump(data[:1000], open("/mnt/workspace/workgroup/haonan/evolution-code/case_1k.json", "w"))